In [52]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [54]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [56]:
dataset = pd.get_dummies(dataset,dtype=int, drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [58]:
# Axis = 0 (Row) By defualt, to specify the Column which is to be deleted Axis=1 should be mentioned. 
dataset = dataset.drop("User ID", axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [60]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [62]:
independent = dataset[["Age","EstimatedSalary","Gender_Male"]]
dependent = dataset["Purchased"]

In [64]:
independent.shape

(400, 3)

In [66]:
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [68]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [38]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

#Here the fit.transform function Standardize the Undefined data to Defined Data and make it to be saved in X_train
X_train = sc.fit_transform(X_train)

#Here the transform function uses the process occurred in fit.transform function for the input X_test and make it to be saved in X_train
X_test = sc.transform(X_test)

X_train

array([[ 0.81203707,  0.7926083 , -0.99250926],
       [-0.96332069, -0.30625786,  1.00754728],
       [ 0.12162016,  0.7926083 ,  1.00754728],
       [-0.96332069,  0.5847147 ,  1.00754728],
       [ 0.31888214,  0.07983025,  1.00754728],
       [ 0.71340608, -1.28632769,  1.00754728],
       [-0.47016576, -0.00926701, -0.99250926],
       [-1.75236858,  0.3768211 , -0.99250926],
       [-0.66742773,  0.13922842,  1.00754728],
       [ 0.41751312,  0.31742293,  1.00754728],
       [-0.27290378,  0.07983025, -0.99250926],
       [-0.47016576,  2.36665983, -0.99250926],
       [ 0.22025115,  0.05013116,  1.00754728],
       [ 1.30519201,  2.27756258, -0.99250926],
       [ 0.81203707,  0.28772385,  1.00754728],
       [-0.27290378,  0.16892751,  1.00754728],
       [ 0.02298918, -0.54385055,  1.00754728],
       [-0.1742728 ,  0.16892751, -0.99250926],
       [-0.07564181,  0.25802476, -0.99250926],
       [ 0.02298918, -0.24685969,  1.00754728],
       [ 2.19287089,  1.14899733, -0.992

In [70]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV #Here cv refers to Cross Validation

#Creating a Dictionary for Playable Parameters
param_grid = {'solver':['newton-cg','lbfgs','liblinear','saga'], 'penalty':['l2']}

#if refit = false then final model will be considered as Last Combination of parameter_grid like (linear,3000,scale)
#n_jobs = -1 refers to Run all the Models Parallely
#refit = True -> Best Value from the GridSearch will be saved in Grid for Model Creation
#scoring = 'f1_weighted' -> It is used for both Balanced and Imbalanced data and can be taken into account for validation instead of Precision/Recall/Accuracy
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3, n_jobs = -1, scoring = 'f1_weighted')

#All the above steps are called "Procedure Load"

#Model Creation
grid.fit(X_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda\Lib\site-packages\scipy\optimize\_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
C:\Anaconda\Lib\site-packages\sklearn\utils\optimize.py:74: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [72]:
result = grid.cv_results_

y_pred = grid.predict(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[79  6]
 [ 9 40]]
              precision    recall  f1-score   support

           0       0.90      0.93      0.91        85
           1       0.87      0.82      0.84        49

    accuracy                           0.89       134
   macro avg       0.88      0.87      0.88       134
weighted avg       0.89      0.89      0.89       134



In [51]:
#clf_report.__add__

<method-wrapper '__add__' of str object at 0x00000202A6549000>

In [39]:
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

Age: 35
BMI: 40
Sex Male 0 or 1: 1


In [41]:
grid.predict([[age_input,salary_input,sex_male_input]])

C:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0], dtype=int64)

In [43]:
#dir(clf_report)

In [49]:
# Naming the Model with its extension .sav
import pickle
filename = "finalized_model_Logistic_classification_Assignment.sav"

#Model will be saved in respective path, once after the below Code Execution
# Keyword - wb refers to "write binary"
pickle.dump(classifier,open(filename,'wb'))

In [51]:
# Load the Saved Model and check the same to send it for Phase-02 (Deployment)
# # Keyword - rb refers to "read binary"

loaded_model=pickle.load(open("finalized_model_Logistic_classification_Assignment.sav",'rb'))
result = loaded_model.predict([[age_input,salary_input,sex_male_input]]) 
result

C:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0], dtype=int64)